# Introduction

### Imports

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Imports</a></span></li></ul></li></ul></li></ul></div>

In [20]:
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd
from collections import Counter
import re
import string

In [1]:
class SurflineSpider(scrapy.Spider):
    name = "CN"
    
    # Here is where we insert our API call.
    start_urls = [
        'http://www.calnewport.com/blog/'
        ]

    def parse(self, response):
        # Iterate over every <article> element on the page.
        for article in response.xpath('/html/body/div[2]/div/div[1]/article'):  
           # paragraph_list = []
            #for paragraph in article.xpath('/p'):  
                #paragraph_list.extend(paragraph.xpath('p/text()'))
            # Yield a dictionary with the values we want.
            yield {
                # This is the code to choose what we want to extract
                # You can modify this with other Xpath expressions to extract other information from the site
                'title': article.xpath('h2/a/text()').extract_first(),
                'date': article.xpath('small/text()').extract_first(),
                'text': article.xpath('p/text()').extract(),

            }
                    
        # Getting the information needed to continue to the next ten entries.
        next_page = response.xpath('continue/@lhcontinue').extract_first()
        
        # Recursively calling the spider to process the next ten entries, if they exist.
        if next_page is not None:
            next_page = '{}&lhcontinue={}'.format(self.start_urls[0],next_page)
            yield scrapy.Request(next_page, callback=self.parse)
            
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'cn.json',
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

process.crawl(SurflineSpider)
process.start()

In [65]:
df = pd.read_json('cn.json')
df['text'] = df['text'].apply(lambda x: ''.join(x))

In [68]:
pattern = '[!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~]'

df['count_text'] = df['text'].apply(lambda x: re.sub(pattern, "", x).split(' '))

df['count_text'] = df['count_text'].apply(lambda x: Counter(x))

df['count_text'][0]